---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [69]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import roc_auc_score
    
    train = pd.read_csv('train.csv', encoding = "ISO-8859-1", parse_dates =[14, 15, 28])
    train = train.set_index('ticket_id')
    test = pd.read_csv('test.csv')
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    
    address = address.set_index('address').join(latlons.set_index('address'), how = 'left')
    train = train.join(address.set_index('ticket_id'))
    test = test.set_index('ticket_id').join(address.set_index('ticket_id'))
    train = train[(train['compliance'] == 0.0) | (train['compliance'] == 1.0)]
    
    train.drop(['violation_zip_code', 'grafitti_status', 'payment_date', 'payment_status', 'hearing_date', 
                           'ticket_issued_date', 'payment_amount', 'payment_status', 
                           'balance_due', 'collection_status', 'compliance_detail'], axis = 1, inplace=True)
    
    train['mailing_address_str_number'].fillna(value=0, inplace=True)
    train['fine_amount'].fillna(value=0, inplace=True)
    train = train[train['country'] == 'USA']

    for col in train.select_dtypes(include=['object']).columns:
        train[col].fillna(value='None', inplace=True)
        train[col] = pd.Categorical(train[col], categories=train[col].unique()).codes
    
    train.fillna(0, inplace = True)
    
    #train_test_split
    X = train.drop(['compliance'], axis = 1)
    y = train.loc[:, 'compliance']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    
    clf = RandomForestClassifier()
    grid_values = {'n_estimators': [10, 100], 'max_depth': [3, 50]}
    grid_rf_auc = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    grid_rf_auc.fit(X_train, y_train)
    print('Model best parameter (max. AUC): ', grid_rf_auc.best_params_)
    print('Model score (AUC): ', grid_rf_auc.best_score_)
    
    train_columns = train.columns.values.tolist()
    train_columns.remove('compliance')
    test = test[train_columns]
    
    for col in test.select_dtypes(include=['object']).columns:
        test[col].fillna(value='None', inplace=True)
        test[col] = pd.Categorical(test[col], categories=test[col].unique()).codes
    test.fillna(0, inplace = True)
    X_predict = grid_rf_auc.predict_proba(test)
    result = pd.Series(data = X_predict[:,1], index = test.index, dtype='float32')
    return result

blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Model best parameter (max. AUC):  {'max_depth': 50, 'n_estimators': 100}
Model score (AUC):  0.831603006994


ticket_id
284932    0.270
285362    0.200
285361    0.410
285338    0.170
285346    0.190
285345    0.190
285347    0.220
285342    0.500
285530    0.170
284989    0.140
285344    0.290
285343    0.240
285340    0.220
285341    0.390
285349    0.200
285348    0.230
284991    0.120
285532    0.270
285406    0.150
285001    0.200
285006    0.160
285405    0.110
285337    0.090
285496    0.200
285497    0.130
285378    0.120
285589    0.170
285585    0.360
285501    0.200
285581    0.180
          ...  
376367    0.010
376366    0.020
376362    0.200
376363    0.220
376365    0.080
376364    0.100
376228    0.140
376265    0.020
376286    0.220
376320    0.085
376314    0.110
376327    0.340
376385    0.360
376435    0.670
376370    0.720
376434    0.240
376459    0.190
376478    0.120
376473    0.110
376484    0.030
376482    0.050
376480    0.060
376479    0.060
376481    0.060
376483    0.040
376496    0.060
376497    0.060
376499    0.400
376500    0.250
369851    0.690
dtype: float32